# Clasificacion de IDC con CNN

## Librerias y Paquetes

In [1]:
### Hay muchas librerias que tal vez no estoy usando falta limpiar

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR, CyclicLR
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

from glob import glob  #El módulo glob puede obtener la ruta completa de todos los archivos elegibles de acuerdo con la ruta especificada
from skimage.io import imread
from os import listdir

import time  #contiene una serie de funciones relacionadas con la medición del tiempo
import copy  #permite crear copias de distintos objetos de Python
from tqdm import tqdm_notebook as tqdm  #Instantly make your loops show a smart progress meter

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.utils import get_file
from tensorflow import keras
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neighbors import DistanceMetric
from imblearn.metrics import sensitivity_score, specificity_score

import os
import zipfile
import random

### Configuraciones

In [2]:
run_training = False
retrain = False
find_learning_rate = False

## Cargamos el Data Set

Aqui he cargado a mano una parte, por lo que es muy prbable que este sesgado y esto dificulte el aprendizaje, lo primero que hay que hacer es generar un codigo que extraiga una muestra aleatorea.


JOSEMA: Cambie la url del dataset para mi pc, tu tendras que hacer lo mismo

In [3]:
files = listdir("C:/Users/juanS/OneDrive/Escritorio/Master IMF/TFM/TFM DS/Data Set/IDC_regular_ps50_idx5")
len(files)

279

In [4]:
files[0:10]

['10253',
 '10254',
 '10255',
 '10256',
 '10257',
 '10258',
 '10259',
 '10260',
 '10261',
 '10262']

In [5]:
folder = files

In [6]:
base_path = "C:/Users/juanS/OneDrive/Escritorio/Master IMF/TFM/TFM DS/Data Set/IDC_regular_ps50_idx5"

Lamentablemente, el número de pacientes es bastante pequeño, por lo que se debe prestar mucha atención al sobreajuste de los datos.

Para contrarestar esta desventaja, procedemos a realizar cortes de 40x40 sobre la imagen del paciente para de esta forma aumentar eficientemente el numero de imagenes que disponemos para nuestro algoritmo.

In [7]:
total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + "/" + patient_id
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        total_images += len(subfiles)

In [8]:
total_images

277524

Nuestro algoritmo debe decidir si un parche de imagen contiene IDC o no. 
No todo el corte de tejido del paciente, sino los parches individuales, es decir que si un parche o corte posee IDC entonces ese usuario posee IDC.

**IDC (+) se considera Cancer**

**IDC (-) Se considera normal**

 

Sería importante **almacenar la ruta de cada imagen**. De esta manera evitamos almacenar cada pixel de cada imagen. 
Si almacenamos el path, podemos cargar lotes de imágenes por 'paciente_id' y el 'target'.

### Creamos el Data Set con el que experimentaremos

In [9]:
data = pd.DataFrame(index=np.arange(0, total_images),
                    columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + "/" + patient_id
    for c in [0, 1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.iloc[k]["path"] = class_path + image_path
            data.iloc[k]["target"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1

data.head()

,patient_id,path,target
0,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,0
1,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,0
2,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,0
3,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,0
4,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,0


## Marco de datos

Tenemos que **extraer todas las coordenadas de los parches de imagen** que están almacenados en los nombres de las imágenes. **Entonces podemos usar las coordenadas para reconstruir todo el tejido mamario de una paciente**. De esta manera, también **podemos explorar cómo se ve el tejido enfermo en comparación con los sanos**. Para simplificar esta tarea, escribamos un método que tome un paciente y genere un marco de datos con coordenadas y objetivos.

In [10]:
def extract_coords(df):
    coord = df.path.str.rsplit("_", n=4, expand=True)
    coord = coord.drop([0, 1, 4], axis=1)
    coord = coord.rename({2: "x", 3: "y"}, axis=1)
    coord.loc[:, "x"] = coord.loc[:,
                                  "x"].str.replace("x", "",
                                                   case=False).astype(np.int)
    coord.loc[:, "y"] = coord.loc[:,
                                  "y"].str.replace("y", "",
                                                   case=False).astype(np.int)
    df.loc[:, "x"] = coord.x.values
    df.loc[:, "y"] = coord.y.values
    return df


def get_cancer_dataframe(patient_id, cancer_id):
    path = base_path + "/" + patient_id + "/" + cancer_id
    files = listdir(path)
    dataframe = pd.DataFrame(files, columns=["filename"])
    path_names = path + "/" + dataframe.filename.values
    dataframe = dataframe.filename.str.rsplit("_", n=4, expand=True)
    dataframe.loc[:, "target"] = np.int(cancer_id)
    dataframe.loc[:, "path"] = path_names
    dataframe = dataframe.drop([0, 1, 4], axis=1)
    dataframe = dataframe.rename({2: "x", 3: "y"}, axis=1)
    dataframe.loc[:, "x"] = dataframe.loc[:, "x"].str.replace(
        "x", "", case=False).astype(np.int)
    dataframe.loc[:, "y"] = dataframe.loc[:, "y"].str.replace(
        "y", "", case=False).astype(np.int)
    return dataframe

def get_patient_dataframe(patient_id):
    df_0 = get_cancer_dataframe(patient_id, "0")
    df_1 = get_cancer_dataframe(patient_id, "1")
    patient_df = df_0.append(df_1)
    return patient_df

## Visualización del tejido mamario 

Esta parte es un poco complicada, tenemos que extraer todas las coordenadas de los parches de imagen que están almacenados en los nombres de las imágenes.

Luego, podemos usar las coordenadas para reconstruir todo el tejido mamario de una paciente. De esta manera también podemos explorar cómo se ve el tejido enfermo en comparación con los sanos. 

**Para simplificar esta tarea, escribimos un método que tome a un paciente y dé como resultado un marco de datos con coordenadas y objetivos.**

In [11]:
example = get_patient_dataframe(data.patient_id.values[0])
example.head()

,x,y,target,path
0,1001,1001,0,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...
1,1001,1051,0,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...
2,1001,1101,0,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...
3,1001,1151,0,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...
4,1001,1201,0,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...


ahora tenemos las coordenadas de cada parche, su ruta para cargar la imagen y su información de destino.

## Visualización de las imágenes del tejido mamario 

Ahora es el momento de ir un paso más profundo con nuestra EDA.

Dadas las coordenadas de los parches de imagen, podríamos intentar reconstruir la imagen de tejido completo (no solo los objetivos).

# Setting up the machine learning workflow

In [12]:
BATCH_SIZE = 32  # tamaño de muestra (lote)
NUM_CLASSES = 2

OUTPUT_PATH = ""
# MODEL_PATH = "C:/Users/juanS/OneDrive/Escritorio/Master IMF/TFM/TFM DS/Data Set/archive"
# LOSSES_PATH = "C:/Users/juanS/OneDrive/Escritorio/Master IMF/TFM/TFM DS/Data Set/archive"

In [13]:
# Para que los resultados sean replicables
torch.manual_seed(0)
np.random.seed(0)

### Continuo trabajando sobre el DF, no el que usamos para la visualizacion de todo el tejido

In [14]:
data.target = data.target.astype(np.int)

In [15]:
data.target = data.target.astype(np.bool)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277524 entries, 0 to 277523
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   patient_id  277524 non-null  object
 1   path        277524 non-null  object
 2   target      277524 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 6.6+ MB


Por lo general, asigno el 80% del conjunto de datos para entrenamiento y el otro 20% para pruebas. En este caso, como se mencionó anteriormente, la validación es crucial. Por lo tanto, asignaré el 70% para el entrenamiento del modelo , el 15% para las pruebas y el 15% para la validación.


JOSEMA: En la siguiente linea me quede con los primeros 50 pacientes, ahi puedes modificar segun la cantidad que quieras

In [17]:
patients = data.patient_id.unique()[0:50]

In [18]:
train_ids, sub_test_ids = train_test_split(patients,
                                           test_size=0.3,
                                           random_state=0)
test_ids, dev_ids = train_test_split(sub_test_ids,
                                     test_size=0.5,
                                     random_state=0)

So far we can't stratify on the targets as we are splitting on patient ids. If we would like to include some target information we would need to create a feature that allows us to generate some balance.

In [19]:
print(
    len(train_ids) / patients.shape[0] * 100,
    len(dev_ids) / patients.shape[0] * 100,
    len(test_ids) / patients.shape[0] * 100)

70.0 16.0 14.000000000000002


In [20]:
print(len(train_ids), len(dev_ids), len(test_ids))

35 8 7


## //////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [21]:
# Validation set is used for determining the parameters of the model

train_df = data.loc[data.patient_id.isin(train_ids), :].copy()
test_df = data.loc[data.patient_id.isin(test_ids), :].copy()
dev_df = data.loc[data.patient_id.isin(dev_ids), :].copy()

# extract cords esta definida mas arriba
train_df = extract_coords(train_df)
test_df = extract_coords(test_df)
dev_df = extract_coords(dev_df)

In [22]:
train_df.head()

,patient_id,path,target,x,y
0,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,False,1001,1001
1,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,False,1001,1051
2,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,False,1001,1101
3,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,False,1001,1151
4,10253,C:/Users/juanS/OneDrive/Escritorio/Master IMF/...,False,1001,1201


In [23]:
# loading data
#train_metadata_filename = "train.csv"
#valid_metadata_filename = "valid.csv"
# load CSV files as DataFrames
#df_train = pd.read_csv(train_metadata_filename)
#df_valid = pd.read_csv(valid_metadata_filename)

#----------------------------------------------------------------------------------------------------

n_training_samples = len(train_df)
n_validation_samples = len(dev_df)
print("Number of training samples:", n_training_samples)
print("Number of validation samples:", n_validation_samples)
train_ds1 = tf.data.Dataset.from_tensor_slices(
    (train_df["path"], train_df["target"]))
valid_ds1 = tf.data.Dataset.from_tensor_slices(
    (dev_df["path"], dev_df["target"]))

## from_tensor_slices() method, we can get the slices of an array in the form of objects by using tf. data -- aplica una transformacion

Number of training samples: 44570
Number of validation samples: 8755


## CLAHE 

JOSEMA: annadi el filtro clahe que  supuestamente elimina el ruido en las imagenes

Se utiliza para eliminar el ruido en la imagen

In [32]:
#pip install cv2
#pip install tf_clahe

SyntaxError: invalid syntax (<ipython-input-32-d07777a756dc>, line 2)

In [35]:
#import cv2
#import tf_clahe
#import tf_clahe
import tensorflow as tf

In [36]:
def improve_contrast_image_using_clahe(bgr_image: np.array) -> np.array:
    hsv = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    hsv_planes = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv_planes = (hsv_planes[0], hsv_planes[1], clahe.apply(hsv_planes[2]))
    hsv = cv2.merge(hsv_planes)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

import tf_clahe

## Modelo

JOSEMA: en decode_img annadi lo del clahe que mencionaba arriba y normalize la imagen (si lo quieres quitar comentas las lineas que dicen JOSEMA)


In [37]:
# preprocess data
normalization_layer = tf.keras.layers.Rescaling(1./255)

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3) # canales rgb
    img = tf_clahe.clahe(img)  ## JOSEMA
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [299, 299]) ## este tamaño es requerido por la red inception V3
    # resize the image to the desired size.
    return normalization_layer(img)  #JOSEMA
    # return normalization_layer(img)  



def process_path(filepath, label):
    # load the raw data from the file as a string
    img = tf.io.read_file(filepath)
    img = decode_img(img)
    return img, label


valid_ds = valid_ds1.map(process_path)  # map procesa una transformacion (process_path) al objeto iterable
train_ds = train_ds1.map(process_path)
# test_ds = test_ds
for image, label in train_ds.take(1):
    print("Image shape:", image.shape)
    print("Label:", label.numpy())

Image shape: (299, 299, 3)
Label: False


**El código anterior usa el método map() para ejecutar la función process_path() en cada muestra en ambos conjuntos, básicamente cargará las imágenes, decodificará el formato de la imagen, convertirá los píxeles de la imagen para que estén en el rango [0, 1] y cambiará el tamaño a (299, 299, 3), luego tomamos una imagen e imprimimos su forma:**

InceptionV3 requiere un formato (299, 299) para ser procesado, por eso redimensionamos la imagen a este tamaño

In [38]:
train_ds

<MapDataset shapes: ((299, 299, 3), ()), types: (tf.float32, tf.bool)>

In [39]:
# training parameters
batch_size = 64
optimizer = "adam"


def prepare_for_training(ds,
                         cache=True,
                         batch_size=64,
                         shuffle_buffer_size=100):
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    # shuffle the dataset
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    # Repeat forever
    #ds = ds.repeat()
    # split to batches
    ds = ds.batch(batch_size)
    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds


valid_ds = prepare_for_training(valid_ds,
                                batch_size=batch_size,
                                cache="valid-cached-data")
train_ds = prepare_for_training(train_ds,
                                batch_size=batch_size,
                                cache="train-cached-data")

### Aqui hacemos un poco de ingenieria para preparar los datos para el entrenamiento.

* Batch mejora el aprendizaje, tomando muestras aleatoreas. Esto impide que aprenda de manera sistematica.
* cache(): Since we're making too many calculations on each set, we used cache() method to save our preprocessed dataset into a local cache file, this will only preprocess it the very first time (in the first epoch during training).
* shuffle(): To basically shuffle the dataset, so the samples are in random order.
* repeat(): Every time we iterate over the dataset, it'll keep generating samples for us repeatedly, this will help us during the training.
* batch(): We batch our dataset into 64 or 32 samples per training step.
* prefetch(): This will enable us to fetch batches in the background while the model is training.

In [40]:
# estas clases las definimos para asignarlas como titulos en la siguiente grafica
class_names = ["benign", "malignant"]

In [41]:
class ResidualUnit(tf.keras.layers.Layer):
    
    def get_config(self):
        cfg = super().get_config()
        return cfg 

batch = next(iter(valid_ds))


def show_batch(batch):
    plt.figure(figsize=(12, 12))
    for n in range(25):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(batch[0][n])
        plt.title(class_names[batch[1][n].numpy()].title())
        plt.axis('off')


show_batch(batch)

JOSEMA: Si normalizas antes as imagenes ya no seran visibles, porque tendran valores entre 0 y 1

### Data Augmentation, flip horizontal / verticaly resize. 

The simplest transformations we can do for each image are:

* resizing the images to the desired input shape, performing horizontal and vertical flips.

* Furthermore we need to create a dataset that loads an image patch of a patient, converts it to RGB, performs the augmentation if it's desired and returns the image, the target, the patient id and the image coordinates.



data_augmentation = keras.Sequential(
  [
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal",
                                                 input_shape=(299,                                                           299,
                                                               3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
  ]
)



## //////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

## Definir la estructura del modelo

JOSEMA: Desde aqui hasta que llegue al error hice lo de InceptionV3 guiandome por la doc y lo que ya tenias, el error me dio porque la cnn guarda en cache y se me lleno el disco duro pero a ti te deberia servir. Tiene dos neuronas de salida pero si quieres lo cambias y pones la sigmoid con 1

In [44]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
#import tensorflow_text

In [51]:
import tensorflow_hub as hub
model = hub.load("C:/Users/juanS/-- Trabajos Master --/TFM (ds)/tf2-preview_inception_v3_classification_4")

In [52]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [54]:
inception_v3 = hub.KerasLayer(hub.load("C:/Users/juanS/-- Trabajos Master --/TFM (ds)/tf2-preview_inception_v3_classification_4"), output_shape=[2048], trainable=False)
feature_extractor_layer = hub.KerasLayer(
    inception_v3,
    input_shape=(299, 299, 3),
    trainable=False)

In [56]:
#feature_batch = feature_extractor_layer(image_batch)
#print(feature_batch.shape)

In [57]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(2)
])

In [58]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1001)              23853833  
_________________________________________________________________
dense (Dense)                (None, 2)                 2004      
Total params: 23,855,837
Trainable params: 2,004
Non-trainable params: 23,853,833
_________________________________________________________________


In [59]:
import datetime

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [60]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)


Got `type(handle)`: <class 'tensorflow_hub.keras_layer.KerasLayer'>



Got `type(handle)`: <class 'tensorflow_hub.keras_layer.KerasLayer'>


Epoch 1/10
697/697 [==============================] - 2308s 3s/step - loss: 0.8252 - acc: 0.8273 - val_loss: 0.5559 - val_acc: 0.7540
Epoch 2/10
697/697 [==============================] - 2391s 3s/step - loss: 0.7342 - acc: 0.8425 - val_loss: 0.5637 - val_acc: 0.7540
Epoch 3/10
697/697 [==============================] - 2350s 3s/step - loss: 0.7120 - acc: 0.8458 - val_loss: 0.5830 - val_acc: 0.7540
Epoch 4/10
697/697 [==============================] - 2314s 3s/step - loss: 0.7036 - acc: 0.8480 - val_loss: 0.6055 - val_acc: 0.7540
Epoch 5/10
697/697 [==============================] - ETA: 0s - loss: 0.6994 - acc: 0.8499 

KeyboardInterrupt: 

## //////////////////////////////////////////////

JOSEMA: ya de aqui para abajo es lo mismo que tenias

In [ ]:
#normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
metrics_df

In [ ]:
import pandas as pd
metrics_df = pd.DataFrame(history.history)

## Model Evaluation

In [ ]:
# evaluation
# load testing set

n_testing_samples = len(test_df)

print("Number of testing samples:", n_testing_samples)

test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["path"], test_df["target"]))



def prepare_for_testing(ds, cache=True, shuffle_buffer_size=100):
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()
  ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  return ds

test_ds = test_ds.map(process_path)
test_ds = prepare_for_testing(test_ds, cache="test-cached-data")

In [ ]:
# convert testing set to numpy array to fit in memory (don't do that when testing
# set is too large)
y_test = np.zeros((n_testing_samples,))
X_test = np.zeros((n_testing_samples, 299, 299, 3))
for i, (img, label) in enumerate(test_ds.take(n_testing_samples)):
    print(img.shape, label.shape)
    X_test[i] = img
    y_test[i] = label.numpy()

print("y_test.shape:", y_test.shape)

In [ ]:
print("Evaluating the model...")
loss, accuracy = m.evaluate(X_test, y_test, verbose=0)
print("Loss:", loss, "  Accuracy:", accuracy)

### Accuracy no es una buena medida ya que los datos estan desbalanceados.

In [ ]:
def get_predictions(threshold=None):
  """
  Returns predictions for binary classification given `threshold`
  For instance, if threshold is 0.3, then it'll output 1 (malignant) for that sample if
  the probability of 1 is 30% or more (instead of 50%)
  """
  y_pred = m.predict(X_test)
  if not threshold:
    threshold = 0.5
  result = np.zeros((n_testing_samples,))
  for i in range(n_testing_samples):
    # test melanoma probability
    if y_pred[i][0] >= threshold:
      result[i] = 1
    # else, it's 0 (benign)
  return result

threshold = 0.23
# get predictions with 23% threshold
# which means if the model is 23% sure or more that is malignant,
# it's assigned as malignant, otherwise it's benign
y_pred = get_predictions(threshold)

In [ ]:
def plot_confusion_matrix(y_test, y_pred):
  cmn = confusion_matrix(y_test, y_pred)
  # Normalise
  cmn = cmn.astype('float') / cmn.sum(axis=1)[:, np.newaxis]
  # print it
  print(cmn)
  fig, ax = plt.subplots(figsize=(10,10))
  sns.heatmap(cmn, annot=True, fmt='.2f', 
              xticklabels=[f"pred_{c}" for c in class_names], 
              yticklabels=[f"true_{c}" for c in class_names],
              cmap="Blues"
              )
  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  # plot the resulting confusion matrix
  plt.show()

plot_confusion_matrix(y_test, y_pred)

In [ ]:
sensitivity = sensitivity_score(y_test, y_pred)
specificity = specificity_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("IDC precision:", precision)
print("IDC recall:", recall)
print("IDC Sensitivity:", sensitivity)
print("IDC Specificity:", specificity)
print("IDC f1:", specificity)


* Precision nos da la calidad de la predicción: ¿qué porcentaje de los que hemos dicho que son la clase positiva, en realidad lo son?

* Recall nos da la cantidad: ¿qué porcentaje de la clase positiva hemos sido capaces de identificar?

* F1 combina Precision y Recall en una sola medida

* La Matriz de Confusión indica qué tipos de errores se cometen

### Binary Cross Entropy Cost Function
For batch gradient descent we need to adjust the Binary Cross Entropy(BCE) Loss function to accommodate not just one example but all the examples in a batch. This adjusted Loss function is called the Cost function(also represented by the letter J in neural network literature and some times also called the objective function).

In [ ]:
def plot_roc_auc(y_true, y_pred):
    """
    This function plots the ROC curves and provides the scores.
    """
    # prepare for figure
    plt.figure()
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    # obtain ROC AUC
    roc_auc = auc(fpr, tpr)
    # print score
    print(f"ROC AUC: {roc_auc:.3f}")
    # plot ROC curve
    plt.plot(fpr, tpr, color="blue", lw=2,
                label='ROC curve (area = {f:.2f})'.format(d=1, f=roc_auc))
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curves')
    plt.legend(loc="lower right")
    plt.show()

plot_roc_auc(y_test, y_pred)